# Vehicle routing with Timefold in a Kotlin notebook

In [81]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.5.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.16.1")

## Domain

Create the domain data classes:

### Location

In [82]:
import com.fasterxml.jackson.annotation.JsonFormat

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double)

### Visit

In [83]:
import java.time.Duration

data class Visit(

    val name: String,
    val location: Location,
    val load: Int) {

    override fun toString(): String = name
}

### Vehicle

In [84]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable


@PlanningEntity
data class Vehicle(

    val name: String,
    val homeLocation: Location,
    val capacity: Int) {

    @PlanningListVariable
    var visits: MutableList<Visit> = ArrayList()
    
    // No-arg constructor required for Timefold
    constructor() : this("", Location(0.0, 0.0), 0)

    override fun toString(): String = name
}

## Constraints

In [85]:
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import java.time.Duration

class VehicleRoutingConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            capacity(constraintFactory)
            // Soft constraints
        )
    }
    
    fun capacity(constraintFactory: ConstraintFactory): Constraint {
        // A room can accommodate at most one lesson at the same time.
        return constraintFactory
            // Select each pair of 2 different lessons ...
            .forEachUniquePair(
                Lesson::class.java,
                // ... in the same timeslot ...
                Joiners.equal(Lesson::timeslot),
                // ... in the same room ...
                Joiners.equal(Lesson::room)
            )
            // ... and penalize each pair with a hard weight.
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("school-timetabling", "Room conflict");
    }

}

Line_83.jupyter.kts (22:14 - 31) Not enough information to infer type variable A
Line_83.jupyter.kts (23:17 - 23) Unresolved reference: Lesson
Line_83.jupyter.kts (25:25 - 30) Not enough information to infer type variable A
Line_83.jupyter.kts (25:31 - 37) Unresolved reference: Lesson
Line_83.jupyter.kts (25:39 - 47) Unresolved reference: timeslot
Line_83.jupyter.kts (27:25 - 30) Not enough information to infer type variable A
Line_83.jupyter.kts (27:31 - 37) Unresolved reference: Lesson
Line_83.jupyter.kts (27:39 - 43) Unresolved reference: room

## Schedule

In [86]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import kotlinx.serialization.Transient


@PlanningSolution
data class Schedule(
    
    @PlanningEntityCollectionProperty
    val vehicles: List<Vehicle>,
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val visits: List<Visit>) {

    @PlanningScore
    @Transient
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList())

}

## Read input data

In [87]:
import java.io.File
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()
val problem: Schedule =  mapper.readValue(File("vehicle-routing-data.json"))

## Solve

In [88]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverFactory: SolverFactory<Schedule> = SolverFactory.create(SolverConfig()
        .withSolutionClass(Schedule::class.java)
        .withEntityClasses(Vehicle::class.java)
        .withConstraintProviderClass(VehicleRoutingConstraintProvider::class.java)
        // The solver runs only for 5 seconds on this small dataset.
        // It's recommended to run for at least 5 minutes ("5m") otherwise.
        .withTerminationSpentLimit(Duration.ofSeconds(5)))

println("Solving the problem ...")
val solver: Solver<Schedule> = solverFactory.buildSolver()
val solution: Schedule = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

Solving the problem ...
Solving finished with score (0hard/0soft).


## Visualize the schedule

In [89]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${solution.score}</p>")
    append("<ul>")
    for (vehicle in solution.vehicles) {
        append("<li>${vehicle.name}: ${vehicle.visits.joinToString(", ")}</li>")
    }
    append("</ul>")
})

Score: 0hard/0soft Vehicle A: Anthisnes Vehicle B: Vehicle C: Vehicle D: Vehicle E: Vehicle F:

## Analyze the score

Let's break down the score per constraint:

In [90]:
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreAnalysis = solutionManager.analyze(solution)

The constraintMap must not be empty.
java.lang.IllegalArgumentException: The constraintMap must not be empty.
	at ai.timefold.solver.core.api.score.analysis.ScoreAnalysis.<init>(ScoreAnalysis.java:59)
	at ai.timefold.solver.core.impl.score.director.InnerScoreDirector.buildScoreAnalysis(InnerScoreDirector.java:485)
	at ai.timefold.solver.core.impl.score.director.InnerScoreDirector.buildScoreAnalysis(InnerScoreDirector.java:459)
	at ai.timefold.solver.core.impl.solver.DefaultSolutionManager.lambda$analyze$1(DefaultSolutionManager.java:110)
	at ai.timefold.solver.core.impl.solver.DefaultSolutionManager.callScoreDirector(DefaultSolutionManager.java:74)
	at ai.timefold.solver.core.impl.solver.DefaultSolutionManager.analyze(DefaultSolutionManager.java:109)
	at ai.timefold.solver.core.api.solver.SolutionManager.analyze(SolutionManager.java:114)
	at Line_88_jupyter.<init>(Line_88.jupyter.kts:4)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHand

And visualize it:

In [91]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${scoreAnalysis.score}</p>")
    append("<ul>")
    for (constraint in scoreAnalysis.constraintMap().values) {
        append("<li>${constraint.constraintRef().constraintName}: ${constraint.score.toShortString()}</li>")
    }
    append("</ul>")
})

Cannot invoke "Line_88_jupyter.getScoreAnalysis()" because "this.$$earlierScripts[86]" is null
java.lang.NullPointerException: Cannot invoke "Line_88_jupyter.getScoreAnalysis()" because "this.$$earlierScripts[86]" is null
	at Line_89_jupyter.<init>(Line_89.jupyter.kts:2)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:502)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:486)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.ko

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [92]:
%use kandy

### TODO

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).